In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('churn_modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#preprocessing of the data
data = data.drop(['RowNumber' , 'CustomerId', 'Surname'],axis = 1)

In [5]:
#Encode Categorical Variable
label_encoder_gender = LabelEncoder()

In [6]:
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
#One HHot Encoding for Geography
oneHot_encoder_Geography = OneHotEncoder(handle_unknown = 'ignore')
geo_encoded = oneHot_encoder_Geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = oneHot_encoder_Geography.get_feature_names_out(['Geography']))


In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
#Combinig Onehotencoding with the main data
data = pd.concat([data.drop('Geography',axis = 1),geo_encoded_df], axis = 1)

In [10]:
data.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
#SPlit the data into features and target
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [12]:
X.shape

(10000, 12)

In [13]:
y.shape

(10000,)

In [14]:
X_train, X_test, y_train , y_test = train_test_split(X, y , test_size = 0.2, random_state = 42)

In [15]:
#Scaling these Features
Scaler = StandardScaler()
X_train = Scaler.fit_transform(X_train)
X_test = Scaler.fit_transform(X_test)

In [16]:
#Save the encoder and scalar for later
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(oneHot_encoder_Geography , file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(Scaler, file)

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [18]:
#Build the model
model = Sequential([
    Dense(64, activation = 'relu',input_shape = (X_train.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1) #Output Layer For regression
    
])

C:\Users\titvi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
#Compile the model
model.compile(optimizer = 'adam',loss = 'mean_absolute_error',metrics = ['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


#SetupTensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
tensorboard_callback= TensorBoard(log_dir = log_dir, histogram_freq  =1 )

In [21]:
#Setup EarlY Stopping 
early_stopping_callback = EarlyStopping(monitor = 'val_loss',patience = 10, restore_best_weights = True)

In [22]:
#TRain the model
history = model.fit(
    X_train , y_train,
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 99571.0859 - mae: 99571.0859 - val_loss: 98419.9375 - val_mae: 98419.9375
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 99733.3047 - mae: 99733.3047 - val_loss: 96388.9297 - val_mae: 96388.9297
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97524.2812 - mae: 97524.2812 - val_loss: 91379.5859 - val_mae: 91379.5859
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 89479.6016 - mae: 89479.6016 - val_loss: 83347.9688 - val_mae: 83347.9688
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 81862.8750 - mae: 81862.8750 - val_loss: 73611.2109 - val_mae: 73611.2109
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 72220.6172 - mae: 72220.6172 - val_loss: 64321.7148 - val_mae: 64321.7148
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 63584.3633 - mae: 63584.3633 - val_loss: 57144.6953 - val_mae: 57144.6953
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - 

In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 36168), started 4 days, 12:41:18 ago. (Use '!kill 36168' to kill it.)

In [25]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51128.2070 - mae: 51128.2070
Test MAE : 50265.23828125
